# 🎯 Análise Didática: Chains Especializadas para Turismo

## 📋 Objetivo
Demonstrar **na prática** como diferentes prompts especializados geram respostas únicas para turismo.

## 🔧 O que este notebook faz:
1. **Configura** componentes básicos (LLM, RAG)
2. **Define** 4 templates especializados
3. **Testa** cada chain independentemente 
4. **Compara** resultados lado a lado
5. **Analisa** diferenças qualitativas

## 🎭 4 Personas Especializadas:
- 🗺️ **Roteirista**: Cria itinerários detalhados
- 🚌 **Especialista Logística**: Foca em transporte
- 🏛️ **Guia Local**: Informações culturais  
- 🗣️ **Tradutor**: Comunicação intercultural

## ✅ Por que este approach:
- **Independente**: Não depende de outros arquivos
- **Didático**: Uma função por célula
- **Testável**: Cada chain é testada isoladamente
- **Prático**: Exemplos reais de uso

In [11]:
# 📦 Importações básicas
import os
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

print("✅ Setup básico carregado!")
print("📋 Próximo passo: configurar componentes específicos")

✅ Setup básico carregado!
📋 Próximo passo: configurar componentes específicos


In [12]:
# 🔧 Configurar LLM (obrigatório para chains)
try:
    from langchain_groq import ChatGroq
    
    llm = ChatGroq(
        temperature=0.1,
        model="llama-3.1-8b-instant",
        groq_api_key=os.getenv('GROQ_API_KEY')
    )
    
    print("✅ LLM carregado com sucesso!")
    
except Exception as e:
    print(f"❌ Erro ao carregar LLM: {e}")
    print("💡 Verifique se GROQ_API_KEY está configurada no .env")

✅ LLM carregado com sucesso!


# 🏗️ PARTE 1: Templates Especializados

Cada template define uma **persona** diferente que influencia como o LLM responde às perguntas sobre turismo.

## 🎭 Como funciona a especialização:
1. **Persona**: Define quem está respondendo ("sou um especialista em...")
2. **Contexto**: Informações da base de conhecimento (RAG)  
3. **Instruções**: Como estruturar a resposta
4. **Resultado**: Resposta personalizada para o domínio

In [19]:
# 🗺️ Template 1: ROTEIRO DE VIAGEM
roteiro_template = """
Você é um especialista em planejamento de roteiros turísticos. 

CONTEXTO DA BASE DE CONHECIMENTO:
{contexto}

PERGUNTA DO USUÁRIO:
{pergunta}

INSTRUÇÕES:
- Crie um roteiro estruturado com horários
- Considere proximidade geográfica dos pontos  
- Inclua dicas práticas de tempo e logística
- Seja específico com horários e durações

RESPOSTA:
"""

print("✅ Template ROTEIRO definido!")
print("📋 Foco: Planejamento temporal e espacial")

✅ Template ROTEIRO definido!
📋 Foco: Planejamento temporal e espacial


In [20]:
# 🚌 Template 2: LOGÍSTICA E TRANSPORTE  
logistica_template = """
Você é um especialista em logística de viagem e sistemas de transporte urbano.

CONTEXTO DA BASE DE CONHECIMENTO:
{contexto}

PERGUNTA DO USUÁRIO:
{pergunta}

INSTRUÇÕES:
- Foque em informações de transporte (como chegar)
- Inclua custos aproximados quando possível
- Sugira alternativas de transporte
- Dê dicas de eficiência e segurança
- Mencione horários e frequências

RESPOSTA:
"""

print("✅ Template LOGÍSTICA definido!")
print("📋 Foco: Transporte, custos e mobilidade")

✅ Template LOGÍSTICA definido!
📋 Foco: Transporte, custos e mobilidade


In [21]:
# 🏛️ Template 3: INFORMAÇÕES LOCAIS
info_local_template = """
Você é um guia turístico local expert com anos de experiência.

CONTEXTO DA BASE DE CONHECIMENTO:
{contexto}

PERGUNTA DO USUÁRIO:
{pergunta}

INSTRUÇÕES:
- Forneça informações detalhadas e culturais
- Inclua dados históricos e curiosidades
- Adicione dicas de "local insider" 
- Mencione preços, horários de funcionamento
- Mantenha tom acolhedor e pessoal

RESPOSTA:
"""

print("✅ Template INFO LOCAL definido!")  
print("📋 Foco: Cultura, história e dicas autênticas")

✅ Template INFO LOCAL definido!
📋 Foco: Cultura, história e dicas autênticas


In [22]:
# 🗣️ Template 4: TRADUÇÃO E IDIOMAS
traducao_template = """
Você é um assistente de tradução e comunicação intercultural especializado em turismo.

CONTEXTO DA BASE DE CONHECIMENTO:
{contexto}

PERGUNTA DO USUÁRIO:
{pergunta}

INSTRUÇÕES:
- Forneça traduções precisas e contextualmente adequadas
- Inclua frases úteis para turistas
- Explique diferenças culturais de comunicação  
- Adicione dicas de pronúncia quando relevante
- Sugira expressões locais importantes

RESPOSTA:
"""

print("✅ Template TRADUÇÃO definido!")
print("📋 Foco: Idiomas, frases úteis e cultura")
print("\n🎯 Todos os 4 templates especializados estão prontos!")

✅ Template TRADUÇÃO definido!
📋 Foco: Idiomas, frases úteis e cultura

🎯 Todos os 4 templates especializados estão prontos!


# 🧠 PARTE 2: Configuração RAG (opcional)

**RAG** = Retrieval Augmented Generation (buscar + gerar)

## 💡 Como funciona:
1. **Pergunta** → converter para vetor  
2. **Buscar** documentos similares no Pinecone
3. **Contexto** → fornecer para o LLM
4. **Resposta** mais específica e precisa

## ⚠️ Nota importante:
As chains funcionam **SEM** RAG também! O RAG apenas melhora a precisão.

# 🔧 Configuração Opcional: Componentes RAG

In [23]:
def obter_contexto_rag(pergunta, usar_rag=True):
    """Função RAG simplificada - busca contexto ou retorna texto padrão"""
    
    if not usar_rag:
        return "Contexto geral sobre turismo será usado pelo LLM com seu conhecimento base."
    
    try:
        # Tentar carregar componentes RAG
        from pinecone import Pinecone
        from langchain_huggingface import HuggingFaceEmbeddings
        
        # Configurar Pinecone e embeddings
        pinecone_client = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
        indice = pinecone_client.Index('guia-viagem')
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        
        # Buscar contexto
        pergunta_embedding = embeddings.embed_query(pergunta)
        resultados = indice.query(vector=pergunta_embedding, top_k=2, include_metadata=True)
        
        # Extrair textos relevantes
        contextos = []
        for match in resultados['matches']:
            if match['score'] > 0.3:
                metadata = match.get('metadata', {})
                texto = metadata.get('texto') or metadata.get('text') or metadata.get('content')
                if texto:
                    contextos.append(texto)
        
        if contextos:
            return '\n\n---\n\n'.join(contextos)
        else:
            return "Nenhum contexto específico encontrado. Usando conhecimento base do LLM."
            
    except Exception as e:
        print(f"⚠️ RAG não disponível: {e}")
        return "RAG indisponível. Usando conhecimento base do LLM."

print("✅ Função RAG configurada!")
print("💡 Funciona COM ou SEM Pinecone configurado")

✅ Função RAG configurada!
💡 Funciona COM ou SEM Pinecone configurado


In [24]:
# 🧪 Teste rápido da função RAG
print("🔍 Testando RAG com pergunta sobre Paris...")

contexto_teste = obter_contexto_rag('restaurantes em Paris')
print(f"\n📋 Contexto recuperado ({len(contexto_teste)} chars):")
print(f"'{contexto_teste[:100]}...'")

print("\n✅ Função RAG funcionando! Pronto para usar nas chains.")

🔍 Testando RAG com pergunta sobre Paris...

📋 Contexto recuperado (300 chars):
'Le Potager du Marais: Restaurante vegano tradicional em Paris.
Dica: Compre ingressos para atrações ...'

✅ Função RAG funcionando! Pronto para usar nas chains.

📋 Contexto recuperado (300 chars):
'Le Potager du Marais: Restaurante vegano tradicional em Paris.
Dica: Compre ingressos para atrações ...'

✅ Função RAG funcionando! Pronto para usar nas chains.


# 🎭 PARTE 3: Testando as Chains Especializadas

Agora vamos **testar cada chain** com a mesma pergunta para ver como respondem diferente!

## 🧪 Metodologia:
- **Pergunta única**: "Como visitar o Louvre?"
- **4 respostas**: Uma de cada persona especializada  
- **Análise**: Comparar os focos e estilos de resposta

In [25]:
# 🗺️ Teste 1: CHAIN ROTEIRO
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Configurar chain
pergunta_teste = "Como visitar o Louvre?"
contexto = obter_contexto_rag(pergunta_teste, usar_rag=False)  # Começar sem RAG para simplicidade

roteiro_prompt = PromptTemplate(
    template=roteiro_template,
    input_variables=['contexto', 'pergunta']
)

roteiro_chain = LLMChain(llm=llm, prompt=roteiro_prompt)

# Executar
print("🗺️ RESPOSTA - ESPECIALISTA EM ROTEIROS:")
print("="*50)

resultado_roteiro = roteiro_chain.invoke({
    'contexto': contexto,
    'pergunta': pergunta_teste
})

print(resultado_roteiro['text'])
print("="*50)

/tmp/ipykernel_94927/3059468440.py:14: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  roteiro_chain = LLMChain(llm=llm, prompt=roteiro_prompt)


🗺️ RESPOSTA - ESPECIALISTA EM ROTEIROS:
**Roteiro para visitar o Louvre**

**Duração do roteiro:** 6 horas

**Horário de início:** 9h00

**Localização:** Paris, França (Metro: Palais-Royal – Musée du Louvre)

**Roteiro:**

**9h00 - 9h30: Chegada e entrada no Louvre**

* Chegue ao Museu do Louvre pelo lado sul, onde se encontra a entrada principal.
* Compre o ingresso online com antecedência para evitar filas.
* A entrada é gratuita para menores de 18 anos e para todos os visitantes aos domingos e terças-feiras.

**9h30 - 11h00: Visita ao Salão de Apoio**

* Inicie a visita pelo Salão de Apoio, onde se encontram obras-primas como a "Vênus de Milo" e a "Diana de Versalhes".
* Aproveite para observar a arquitetura do salão e a decoração.

**11h00 - 12h30: Visita ao Salão de Apolo**

* Dirija-se ao Salão de Apolo, onde se encontram obras-primas como a "Noite" de Rembrandt e a "O Banho Turco" de François Boucher.
* Aproveite para observar a arquitetura do salão e a decoração.

**12h30 - 13h

In [26]:
# 🚌 Teste 2: CHAIN LOGÍSTICA
logistica_prompt = PromptTemplate(
    template=logistica_template,
    input_variables=['contexto', 'pergunta']
)

logistica_chain = LLMChain(llm=llm, prompt=logistica_prompt)

# Executar
print("🚌 RESPOSTA - ESPECIALISTA EM LOGÍSTICA:")
print("="*50)

resultado_logistica = logistica_chain.invoke({
    'contexto': contexto,
    'pergunta': pergunta_teste
})

print(resultado_logistica['text'])
print("="*50)

🚌 RESPOSTA - ESPECIALISTA EM LOGÍSTICA:
Visitar o Louvre é uma experiência incrível, mas pode ser um desafio para quem não conhece a cidade de Paris. Aqui estão algumas dicas para ajudá-lo a chegar ao Louvre de forma eficiente e segura:

**Transporte Público**

A melhor opção para chegar ao Louvre é usar o transporte público. Você pode pegar o metrô linha 1 (linha amarela) e descer na estação "Louvre - Rivoli" ou "Palais-Royal - Musée du Louvre". A estação é bem sinalizada e fácil de encontrar.

**Custo aproximado:** 1,90€ por bilhete único (tarifa padrão do metrô de Paris)

**Alternativas de transporte:**

* **Ônibus:** Você também pode pegar o ônibus nº 21, 24, 27, 68 ou 95, que passam pela Rue de Rivoli, perto do Louvre.
* **Taxi:** Se você preferir uma opção mais rápida e confortável, pode pegar um táxi. No entanto, isso pode ser mais caro (cerca de 10-15€).
* **Bicicleta:** Se você está se sentindo aventureiro, pode alugar uma bicicleta e pedalar até o Louvre. Existem várias estaç

In [27]:
# 🏛️ Teste 3: CHAIN INFORMAÇÕES LOCAIS
info_local_prompt = PromptTemplate(
    template=info_local_template,
    input_variables=['contexto', 'pergunta']
)

info_local_chain = LLMChain(llm=llm, prompt=info_local_prompt)

# Executar  
print("🏛️ RESPOSTA - GUIA TURÍSTICO LOCAL:")
print("="*50)

resultado_local = info_local_chain.invoke({
    'contexto': contexto,
    'pergunta': pergunta_teste
})

print(resultado_local['text'])
print("="*50)

🏛️ RESPOSTA - GUIA TURÍSTICO LOCAL:
Bem-vindo ao Louvre, um dos museus mais famosos do mundo! Estou aqui para ajudá-lo a explorar essa joia cultural e histórica. Como um guia turístico local, posso compartilhar com você as melhores dicas e segredos para aproveitar ao máximo sua visita.

**História e Importância**

O Louvre foi fundado em 1793, durante a Revolução Francesa, e desde então se tornou um dos principais museus do mundo. O edifício original foi construído no século XIII como um castelo real, e ao longo dos séculos, foi ampliado e transformado em um museu. Hoje, o Louvre é lar de mais de 550.000 obras de arte, incluindo a famosa "Mona Lisa" de Leonardo da Vinci.

**Dicas de "Local Insider"**

1. **Chegue cedo**: O Louvre é um dos museus mais visitados do mundo, então é importante chegar cedo para evitar as multidões. A entrada gratuita é oferecida das 9h30 às 18h, mas é recomendável chegar antes das 10h para evitar as filas.
2. **Use o aplicativo**: O Louvre oferece um aplicat

In [28]:
# 🗣️ Teste 4: CHAIN TRADUÇÃO
pergunta_traducao = "Como pedir informações sobre o Louvre em francês?"

traducao_prompt = PromptTemplate(
    template=traducao_template,
    input_variables=['contexto', 'pergunta']
)

traducao_chain = LLMChain(llm=llm, prompt=traducao_prompt)

# Executar
print("🗣️ RESPOSTA - ESPECIALISTA EM TRADUÇÃO:")
print("="*50)

resultado_traducao = traducao_chain.invoke({
    'contexto': contexto,
    'pergunta': pergunta_traducao
})

print(resultado_traducao['text'])
print("="*50)

🗣️ RESPOSTA - ESPECIALISTA EM TRADUÇÃO:
Excelente pergunta!

Para pedir informações sobre o Louvre em francês, você pode usar as seguintes frases:

**Perguntas básicas**

* "Pouvez-vous me donner des informations sur le Louvre?" (Pode me dar informações sobre o Louvre?)
* "Qu'est-ce que je peux voir au Louvre?" (O que posso ver no Louvre?)
* "Comment puis-je accéder au Louvre?" (Como posso acessar o Louvre?)

**Informações específicas**

* "Quels sont les horaires d'ouverture du Louvre?" (Quais são os horários de abertura do Louvre?)
* "Quel est le coût de l'entrée au Louvre?" (Qual é o custo da entrada no Louvre?)
* "Y a-t-il des visites guidées au Louvre?" (Há visitas guiadas no Louvre?)

**Dicas de pronúncia**

* "Louvre" é pronunciado como "LOO-vruh" (com um tom suave no final)
* "Pouvez-vous" é pronunciado como "poo-vay" (com um tom suave no final)
* "Qu'est-ce que" é pronunciado como "keh-seh" (com um tom suave no final)

**Diferenças culturais de comunicação**

* Nos países de l

# 🔍 PARTE 4: Análise Comparativa

## 📊 O que observar nas respostas acima:

### 🗺️ Chain ROTEIRO:
- **Foco**: Horários, sequência, tempo de visita
- **Estrutura**: Cronograma organizado  
- **Exemplo**: "9:00 - chegada", "10:00 - entrada"

### 🚌 Chain LOGÍSTICA:  
- **Foco**: Como chegar, transporte, custos
- **Estrutura**: Opções de mobilidade
- **Exemplo**: "Metrô linha 1", "Ônibus 21"

### 🏛️ Chain INFO LOCAL:
- **Foco**: História, cultura, curiosidades  
- **Estrutura**: Contexto enriquecido
- **Exemplo**: "Construído em...", "Obra famosa..."

### 🗣️ Chain TRADUÇÃO:
- **Foco**: Frases úteis, comunicação
- **Estrutura**: Traduções práticas
- **Exemplo**: "Où est le Louvre?" = "Onde fica o Louvre?"


In [31]:
# 🔄 Teste EXTRA: Com RAG ativado (se Pinecone estiver configurado)

print("🧠 Testando a mesma pergunta COM contexto RAG...")

contexto_rag = obter_contexto_rag("Como visitar o Louvre?", usar_rag=True)

if "RAG indisponível" not in contexto_rag:
    print("✅ RAG funcionando! Testando com contexto específico:")
    resultado_com_rag = roteiro_chain.invoke({
        'contexto': contexto_rag,
        'pergunta': "Como visitar o Louvre?"
    })
    print("="*50)
    print("🧠 RESPOSTA COM RAG (mais específica):")
    print(resultado_com_rag['text'][:300] + "...")
    print("="*50)
else:
    print("⚠️ RAG não configurado. As chains funcionam com conhecimento base do LLM!")
    print("💡 Para ativar RAG: configure Pinecone e execute 02-Popular_Pinecone.ipynb")

🧠 Testando a mesma pergunta COM contexto RAG...
✅ RAG funcionando! Testando com contexto específico:
✅ RAG funcionando! Testando com contexto específico:
🧠 RESPOSTA COM RAG (mais específica):
**Roteiro de Visita ao Museu do Louvre**

**Duração:** 6 horas (com tempo para almoço e descanso)

**Horários:**

* 9h00: Chegada ao Museu do Louvre
* 9h15: Entrada no Museu do Louvre (com entrada prioritária para evitar filas)
* 10h00: Visita ao Pavilhão Denon (onde se encontra a Mona Lisa)
* 11h30...
🧠 RESPOSTA COM RAG (mais específica):
**Roteiro de Visita ao Museu do Louvre**

**Duração:** 6 horas (com tempo para almoço e descanso)

**Horários:**

* 9h00: Chegada ao Museu do Louvre
* 9h15: Entrada no Museu do Louvre (com entrada prioritária para evitar filas)
* 10h00: Visita ao Pavilhão Denon (onde se encontra a Mona Lisa)
* 11h30...


In [32]:
# 📊 Análise quantitativa rápida
print("📊 COMPARAÇÃO QUANTITATIVA DAS RESPOSTAS:")
print(f"🗺️ Roteiro: {len(resultado_roteiro['text'])} caracteres")
print(f"🚌 Logística: {len(resultado_logistica['text'])} caracteres") 
print(f"🏛️ Info Local: {len(resultado_local['text'])} caracteres")
print(f"🗣️ Tradução: {len(resultado_traducao['text'])} caracteres")

print("\n✅ CONCLUSÃO PRÁTICA:")
print("• Cada template gera respostas com FOCOS diferentes")
print("• Mesmo LLM, prompts diferentes = outputs especializados") 
print("• Sistema modular: fácil adicionar novas especialidades")
print("• RAG opcional: melhora precisão mas não é obrigatório")

📊 COMPARAÇÃO QUANTITATIVA DAS RESPOSTAS:
🗺️ Roteiro: 2291 caracteres
🚌 Logística: 2049 caracteres
🏛️ Info Local: 3255 caracteres
🗣️ Tradução: 1641 caracteres

✅ CONCLUSÃO PRÁTICA:
• Cada template gera respostas com FOCOS diferentes
• Mesmo LLM, prompts diferentes = outputs especializados
• Sistema modular: fácil adicionar novas especialidades
• RAG opcional: melhora precisão mas não é obrigatório


# 🎓 Conclusões Acadêmicas

## 💡 Principais aprendizados:

### 🎯 **Especialização via Prompt Engineering**:
- **Persona**: Define "quem" está respondendo (especialista, guia, tradutor)
- **Instruções**: Direcionam o FOCO da resposta (horários vs. transporte vs. cultura)  
- **Resultado**: Respostas únicas mesmo com o MESMO LLM base

### 🏗️ **Arquitetura Modular**:
- **4 Templates** independentes = 4 especialidades diferentes
- **Fácil expansão**: Adicionar novos templates = novas especialidades
- **Reutilização**: Mesmo código base, prompts diferentes

### 🧠 **RAG como Enhancement**:
- **Funciona SEM**: LLM tem conhecimento base sobre turismo
- **Melhora COM**: Contexto específico da base de conhecimento
- **Flexível**: Sistema adapta-se se RAG não estiver disponível

### 🚀 **Aplicações Práticas**:
- **Chatbots especializados** para diferentes domínios
- **Assistentes virtuais** com múltiplas personalidades
- **Sistemas de recomendação** contextualizados
- **Automação de atendimento** por especialidade

## ✅ **Próximos passos sugeridos**:
1. Testar com suas próprias perguntas
2. Criar novos templates para outras especialidades
3. Integrar com interface web (Streamlit/Gradio)
4. Expandir base de conhecimento no Pinecone